# Test SkOpt with ask () tell() and Neuron Group

### Example from their git

In [1]:
import numpy as np

from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod



In [2]:
from skopt import Optimizer
from skopt.space import Real
from sklearn.externals.joblib import Parallel, delayed
# example objective taken from skopt
from skopt.benchmarks import branin

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


In [3]:
optimizer = Optimizer(
    dimensions=[Real(-5.0, 10.0), Real(0.0, 15.0)],
    random_state=1,
    base_estimator='gp'
)

In [4]:
for i in range(10): 
    x = optimizer.ask(n_points=4)  # x is a list of n_points points    
    y = Parallel(n_jobs=4)(delayed(branin)(v) for v in x)  # evaluate points in parallel
    print('x', x)
    print('x shape', np.shape(x))
    print('y', y)
    print('y shape', np.shape(y))
    optimizer.tell(x, y)

x [[5.539674886665384, 0.9247759772133487], [0.6533781426078518, 2.1846509309627606], [2.3976321059971877, 13.33388107457551], [-4.122692211373234, 3.835422267410678]]
x shape (4, 2)
y [17.117802423039286, 25.636791759237006, 111.24456644018927, 123.94463659127379]
y shape (4,)
x [[5.253361598114372, 5.019098178016241], [-4.558830399495135, 6.521625092942932], [1.0517974825501595, 11.196568270857433], [-4.341063750925826, 5.52471184343409]]
x shape (4, 2)
y [29.49805986824328, 97.24532144298679, 60.023920779206286, 102.92404054178502]
y shape (4,)
x [[7.455985183109318, 0.20173575763695223], [6.5480183384544315, 4.201808432696337], [6.411163292726274, 1.3168776477532418], [6.035820747326337, 0.0]]
x shape (4, 2)
y [14.96091221397041, 28.78036409352106, 19.567975952807018, 20.519902503043827]
y shape (4,)
x [[7.543548690076646, 0.0], [7.500941007924283, 0.11291580948130737], [3.1589029519796927, 1.551323999490736], [4.536178973049552, 1.4339139689522926]]
x shape (4, 2)
y [14.7430796249

In [5]:
# takes ~ 20 sec to get here
print(min(optimizer.yi))  # print the best objective found

0.3982524854050702


### Same with Separate loops

In [6]:
optimizer = Optimizer(
    dimensions=[Real(-5.0, 10.0), Real(0.0, 15.0)],
    random_state=1,
    base_estimator='gp'
)

In [7]:
n_samples = 1
params = []
candidates = []

for _ in range(n_samples):
    x = optimizer.ask(n_points=10)  # x is a list of n_points points    
    candidates.append(x)


In [50]:
optimizer.ask(20)

[[1.4641220621648614, 6.081135959811592],
 [2.3639360846506214, 4.248174055235343],
 [2.6860447937950562, 3.2912931536673677],
 [3.9606335727961053, 8.20875816653289],
 [3.88706167427406, 9.71258483774376],
 [-2.449031993214122, 2.5113069552301486],
 [1.8060972908965978, 1.6026057996698964],
 [1.2979693956592584, 7.009291517505508],
 [1.7315878945833827, 6.820852659055204],
 [-1.0408148524870753, 6.7329253605862025],
 [2.7820138394813814, 1.1518997790982402],
 [2.113759483186919, 5.460803060286988],
 [4.205063112660705, 5.94930424238128],
 [-0.37752486480505354, 1.4571729273226088],
 [1.109682918022803, 4.582013421622731],
 [-1.8475007550590266, 8.098721233649801],
 [-4.990269198080658, 8.331085823701427],
 [4.066111550333094, 0.2905521352020524],
 [-3.271879589494044, 6.316677754211812],
 [1.6985671830378877, 1.2111951404329326]]

In [8]:
candidates

[[[5.539674886665384, 0.9247759772133487],
  [0.6533781426078518, 2.1846509309627606],
  [2.3976321059971877, 13.33388107457551],
  [-4.122692211373234, 3.835422267410678],
  [2.18434866398996, 10.14131246631503],
  [-0.26536359546881716, 14.474360413114516],
  [-4.635512700781785, 14.054611867944384],
  [7.44081018418634, 2.016921234869212],
  [4.852135388499796, 9.287165289493949],
  [8.747572428177428, 7.5085739238532625]]]

In [9]:
for cand in candidates:
    y = Parallel(n_jobs=1)(delayed(branin)(v) for v in x)  # evaluate points in parallel
    optimizer.tell(cand, y)        
    print(y)

[17.117802423039286, 25.636791759237006, 111.24456644018927, 123.94463659127379, 53.491263803604525, 83.95464222088268, 13.66814170756434, 14.355184112861938, 74.82906435343783, 33.27012035987897]


In [10]:
print(min(optimizer.yi))

13.66814170756434


## Load the Model

### Input

In [11]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [12]:
# Create target current traces
output_traces = 10*nS*input_traces

In [13]:
input = input_traces
output = output_traces

### Parameters 

In [14]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])

In [15]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

In [16]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [17]:
parameters_dict(params)

{'g': array([1.80869973e-08, 1.88373085e-08, 1.88373085e-08]),
 'E': array([0.0250218 , 0.09895599, 0.09895599])}

### Setup The Model for Optimization

In [18]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [19]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.01s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [20]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [21]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [22]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [23]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

#### Record error  
additional differential equation calculating the error

In [24]:
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

# store the state of the network
store()

In [25]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [26]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Ask and Tel with calc_error Function

In [27]:
start_scope()

In [28]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [29]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [30]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [31]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### Ask and Tel with calc_error Function

In [32]:
optimizer = Optimizer(
    dimensions=[Real(-5.0, 5.0), Real(0.0, 10.0)],
    random_state=1,
    base_estimator='gp',
    n_initial_points=100
)

n_samples = 10
params = []
candidates = []

for _ in range(n_samples):
    
    cand = optim.ask()
    candidates.append(cand)
    params.append(list(cand.args))
    
params

In [33]:
n_rounds = 3
n_samples = 10
params = []
candidates = []

In [34]:
for _ in range(n_rounds):
    x = optimizer.ask(n_points=n_samples)  # x is a list of n_points points    
    candidates.append(x)

In [35]:
candidates

[[[2.0264499244435896, 0.6165173181422324],
  [-1.2310812382614325, 1.4564339539751738],
  [-0.06824526266854125, 8.889254049717007],
  [-4.415128140915489, 2.5569481782737853],
  [-0.21043422400669343, 6.760874977543353],
  [-1.843575730312545, 9.64957360874301],
  [-4.757008467187856, 9.369741245296256],
  [3.293873456124226, 1.3446141565794747],
  [1.5680902589998649, 6.191443526329299],
  [4.165048285451618, 5.005715949235508]],
 [[2.0264499244435896, 0.6165173181422324],
  [-1.2310812382614325, 1.4564339539751738],
  [-0.06824526266854125, 8.889254049717007],
  [-4.415128140915489, 2.5569481782737853],
  [-0.21043422400669343, 6.760874977543353],
  [-1.843575730312545, 9.64957360874301],
  [-4.757008467187856, 9.369741245296256],
  [3.293873456124226, 1.3446141565794747],
  [1.5680902589998649, 6.191443526329299],
  [4.165048285451618, 5.005715949235508]],
 [[2.0264499244435896, 0.6165173181422324],
  [-1.2310812382614325, 1.4564339539751738],
  [-0.06824526266854125, 8.8892540497

In [36]:
y = []

for n in range(n_rounds):
    y.append(calc_error(np.array(candidates[n])))

[[ 2.02644992  0.61651732]
 [-1.23108124  1.45643395]
 [-0.06824526  8.88925405]
 [-4.41512814  2.55694818]
 [-0.21043422  6.76087498]
 [-1.84357573  9.64957361]
 [-4.75700847  9.36974125]
 [ 3.29387346  1.34461416]
 [ 1.56809026  6.19144353]
 [ 4.16504829  5.00571595]]
[[ 2.02644992  0.61651732]
 [-1.23108124  1.45643395]
 [-0.06824526  8.88925405]
 [-4.41512814  2.55694818]
 [-0.21043422  6.76087498]
 [-1.84357573  9.64957361]
 [-4.75700847  9.36974125]
 [ 3.29387346  1.34461416]
 [ 1.56809026  6.19144353]
 [ 4.16504829  5.00571595]]
[[ 2.02644992  0.61651732]
 [-1.23108124  1.45643395]
 [-0.06824526  8.88925405]
 [-4.41512814  2.55694818]
 [-0.21043422  6.76087498]
 [-1.84357573  9.64957361]
 [-4.75700847  9.36974125]
 [ 3.29387346  1.34461416]
 [ 1.56809026  6.19144353]
 [ 4.16504829  5.00571595]]


In [37]:
y[0]

array([1.56085412e+00, 3.21480782e+00, 3.68023589e-01, 1.27447253e+02,
       2.02413072e+00, 3.16474127e+02, 1.98665770e+03, 1.96159425e+01,
       9.42596767e+01, 4.34682829e+02])

In [38]:
for n in range(n_rounds):
    optimizer.tell(candidates[n], y[n].tolist());

In [39]:
min(optimizer.yi)  # print the best objective found    

0.3680235886120897

In [40]:
xi = optimizer.Xi

In [41]:
yii = np.array(optimizer.yi)
yii.argmin()

2

In [42]:
xi[yii.argmin()]

[-0.06824526266854125, 8.889254049717007]